<a href="https://colab.research.google.com/github/NULabTMN/ps2-PoojaKShanbhag/blob/master/LanguageModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Your task is to train *character-level* language models. 
You will train unigram, bigram, and trigram character level models on a collection of books from Project Gutenberg. You will then use these trained English language models to distinguish English documents from Brazilian Portuguese documents in the test set.

In [0]:
import pandas as pd

import httpimport

with httpimport.remote_repo(['lm_helper'], 'https://raw.githubusercontent.com/jasoriya/CS6120-PS2-support/master/utils/'):
  from lm_helper import get_train_data, get_test_data

This code loads the training and test data. Each dataset is a list of books. Each book contains a list of sentences, and each sentence contains a list of words. For building a character language model, you should join the words of a sentence together with a space character.

In [0]:
# get the train and test data
train = get_train_data()
test, test_files = get_test_data()

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

def uni_statistics(listOfSentences):
    count_vec_unigram = CountVectorizer(analyzer = 'char', ngram_range = (1,1), lowercase=False)
    uni_count_list = count_vec_unigram.fit_transform(listOfSentences).toarray().sum(axis=0)
    uni_word_list = count_vec_unigram.get_feature_names()

    uni_stat = dict(zip(uni_word_list,uni_count_list))
   # print(len(uni_stat))
    return uni_stat

In [0]:
def bi_statistics(listOfSentences):
    
    count_vec_bigram = CountVectorizer(analyzer = 'char', ngram_range = (2,2), lowercase=False)
    bi_count_list = count_vec_bigram.fit_transform(listOfSentences).toarray().sum(axis=0)
    bi_word_list = count_vec_bigram.get_feature_names()

    bi_stat = dict(zip(bi_word_list,bi_count_list))
  #  print(len(bi_stat))
    
    return bi_stat

In [0]:
def tri_statistics(listOfSentences):
    
    count_vec_trigram = CountVectorizer(analyzer = 'char', ngram_range = (3,3), lowercase=False)
    tri_count_list = count_vec_trigram.fit_transform(listOfSentences).toarray().sum(axis=0)
    tri_word_list = count_vec_trigram.get_feature_names()

    tri_stat = dict(zip(tri_word_list,tri_count_list))

    #print(len(tri_stat))
    return tri_stat

## 1.1
Collect statistics on the unigram, bigram, and trigram character counts.

If your machine takes a long time to perform this computation, you may save these counts to files in your github repository and load them on request. This is not necessary, however.



In [0]:
# Your code here
from sklearn.feature_extraction.text import CountVectorizer

listOfSentences = []
for books in train:
    for sentence in books:
        listOfSentences.append(' '.join(sentence))

In [0]:
uni_stat = uni_statistics(listOfSentences)
total_uni_full = sum(uni_stat.values())

In [0]:
print(total_uni_full)

12010505


In [0]:
bi_stat = bi_statistics(listOfSentences)

In [0]:
tri_stat = tri_statistics(listOfSentences)

In [0]:
import math
#print(len(listOfSentences))
length = math.ceil(len(listOfSentences)*0.8)

In [0]:
#get stats on 80% of training data
u_stat_80 = uni_statistics(listOfSentences[:length])
total_uni_80 = sum(u_stat_80.values())
#print(total_uni_80)

In [0]:
b_stat_80 = bi_statistics(listOfSentences[:length])

In [0]:
t_stat_80 = tri_statistics(listOfSentences[:length])

## 1.2
Calculate the perplexity for each document in the test set using linear interpolation smoothing method. For determining λs for linear interpolation, you can divide the training data into a new training set (80%) and a held-out set (20%), then using grid search method:
Choose ~10 values of λ to test using grid search on held-out data.

Some documents in the test set are in Brazilian Portuguese. Identify them as follows: 
  - Sort by perplexity and set a cut-off threshold. All the documents above this threshold score should be categorized as Brazilian Portuguese. 
  - Print the file names (from `test_files`) and perplexities of the documents above the threshold

    ```
        file name, score
        file name, score
        . . .
        file name, score
    ```

  - Copy this list of filenames and manually annotate them as being correctly or incorrectly labeled as Portuguese.




In [0]:
UNK = 50

In [0]:
def unigram_probability(word,flag):
    if flag:
        return uni_stat.get(word,UNK)/total_uni_full
    else:
        return u_stat_80.get(word,UNK)/total_uni_80

In [0]:
def bigram_probabililty(w1,w2,flag):
    if flag:
        return bi_stat.get(w1+w2,0)/uni_stat.get(w1,UNK)
    else:
        return b_stat_80.get(w1+w2,0)/u_stat_80.get(w1,UNK)

In [0]:
def trigram_probability(w1,w2,w3,flag):
    if flag:
        return tri_stat.get(w1+w2+w3,0)/bi_stat.get(w1+w2,UNK)
    else:
        return t_stat_80.get(w1+w2+w3,0)/b_stat_80.get(w1+w2,UNK)

In [0]:
def getLinearInterpolation(w1,w2,w3, l, flag):
    return (l[0]*trigram_probability(w1,w2,w3,flag)+l[1]*bigram_probabililty(w2,w3,flag)+l[2]*unigram_probability(w3,flag))

In [0]:
def calculateInterpolationEntropy(m,sentence,l,flag):
    entropy = 0
    for i in range(0,len(sentence)-2):
        w1, w2, w3 = sentence[i:i+3]
       # print(w1,w2,w3)
        p = getLinearInterpolation(w1,w2,w3,l,flag)
       # print(p)
        entropy = entropy + np.log2(p)
   # print(entropy*-1)
    entropy = (1/m)*(-entropy)
    return entropy

In [0]:
def perplexityCalc(entropy):
    #print(2**(-entropy))
    return 2**(entropy)

In [0]:
# Your code here
import numpy as np
held_out = listOfSentences[length:]
held_out_doc = ' '.join(held_out)

#print(held_out_doc[:5])
#print(len(held_out))

lambdas = [[0.33,0.33,0.33], [0.4,0.3,0.3], [0.8,0.1,0.1], [0.1,0.8,0.1], [0.2,0.2,0.6], [0.2,0.6,0.2], [0.55,0.25,0.2],
           [0.95,0.025,0.025],[0.3, 0.1, 0.6],[0.3, 0.6, 0.1],[0.1, 0.3, 0.6],[0.5, 0.1, 0.4],[0.49,0.49,0.02],[0.02,0.49,0.49],
           [0.49,0.02,0.49]]

perplexity = []
M = len(held_out)
for l in lambdas:
    e = calculateInterpolationEntropy(M,held_out_doc, l,False)
    perp = perplexityCalc(e)
    perplexity.append(perp)
    
index = perplexity.index(min(perplexity))
finalL = lambdas[index]
#print(perplexity[:20])
print(finalL)

[0.95, 0.025, 0.025]


In [0]:
docSentences = []
for doc in test:
    listOfTestSent = []
    for sentence in doc:
        listOfTestSent.append(' '.join(sentence))
    docSentences.append(' '.join(listOfTestSent))

In [0]:
dictionary = {"document":docSentences, "file name":test_files, "language" : "", "perplexity" : np.nan, "isCorrectLang":False}
dataFrame = pd.DataFrame(dictionary)

for i in range(len(dataFrame)):
    if dataFrame["file name"].iloc[i].endswith('.txt'):
        dataFrame["language"].iloc[i]= 'pr'
    else:
        dataFrame["language"].iloc[i]= 'en'

In [0]:

for i in range(len(dataFrame)):
    eachLen = len(dataFrame["document"].iloc[i])
    e = calculateInterpolationEntropy(eachLen,dataFrame["document"].iloc[i], finalL, True)
    dataFrame["perplexity"].iloc[i] = perplexityCalc(e)

In [0]:
threshold = 32

In [0]:

for i in range(len(dataFrame)):
    if dataFrame['perplexity'].iloc[i] >= threshold and dataFrame['language'].iloc[i] == 'pr' :
        dataFrame['isCorrectLang'].iloc[i] = True
    elif dataFrame['perplexity'].iloc[i] < threshold and dataFrame['language'].iloc[i] == 'en' :
        dataFrame['isCorrectLang'].iloc[i] = True
dataFrame = dataFrame.sort_values(by=['perplexity'])
print(dataFrame[170:220])

                                              document     file name language  \
123  Enrique Jorda , conductor and musical director...          ca32       en   
199  St. Johns , Mich. , April 19 . -- A jury of se...          ca21       en   
47   Resentment welled up yesterday among Democrati...          ca07       en   
206  ( Do start fires one or two hours ahead of tim...          ce14       en   
29   The most surprising thing about the Twenty-sec...          cb25       en   
114  The presidency : talking and listening Though ...          ca42       en   
136  Austin , Texas -- Committee approval of Gov. P...          ca02       en   
188  Knowing specifically what the many feed additi...          ce27       en   
126  City Controller Alexander Hemphill charged Tue...          ca09       en   
196  About 70 North Providence taxpayers made appea...          ca24       en   
41   If the Cardinals heed Manager Gene Mauch of th...          ca15       en   
124  California Democrats th

## 1.3
Build a trigram language model with add-λ smoothing (use λ = 0.1).

Sort the test documents by perplexity and perform a check for Brazilian Portuguese documents as above:

  - Observe the perplexity scores and set a cut-off threshold. All the documents above this threshold score should be categorized as Brazilian Portuguese. 
  - Print the file names and perplexities of the documents above the threshold

  ```
      file name, score
      file name, score
      . . .
      file name, score
  ```

  - Copy this list of filenames and manually annotate them for correctness.

In [0]:
l = 0.1

trigram_vocab = sum(tri_stat.values())
def addKlambaProb(w1,w2,w3):
    return (tri_stat.get(w1+w2+w3,0)+l)/(bi_stat.get(w1+w2,0)+(l*len(tri_stat))) 

In [0]:
# print(tri_stat.values())

In [0]:
def calculateAddKEntropy(sentence,m):
    entropy = 0
    for i in range(0,len(sentence)-2):
        w1, w2, w3 = sentence[i:i+3]
       # print(w1,w2,w3)
        p = addKlambaProb(w1,w2,w3)
        #print(p)
        entropy = entropy + np.log2(p)
    #print(entropy*-1)
    entropy = (1/m)*(-entropy)
    return entropy

In [0]:
# Your code here
test_prep = []
i = 0
for i in range(len(docSentences)):
#     e = calculateInterpolationEntropy(doc,finalL, True)
#     test_perp.append(perplexityCalc(e))
    eachLen = len(dataFrame["document"].iloc[i])
    e = calculateAddKEntropy(dataFrame["document"].iloc[i], eachLen)
    dataFrame["perplexity"].iloc[i] = perplexityCalc(e)

In [0]:
threshold = 40
for i in range(len(dataFrame)):
    if dataFrame['perplexity'].iloc[i] >= threshold and dataFrame['language'].iloc[i] == 'pr' :
        dataFrame['isCorrectLang'].iloc[i] = True
    elif dataFrame['perplexity'].iloc[i] < threshold and dataFrame['language'].iloc[i] == 'en' :
        dataFrame['isCorrectLang'].iloc[i] = True
dataFrame2 = dataFrame.copy()
dataFrame2 = dataFrame2.sort_values(by=['perplexity'])
print(dataFrame2[170:220])

                                              document     file name language  \
154  Philadelphia , Jan. 23 -- Nick Skorich , the l...          ca14       en   
33   Mischa Elman shared last night's Lewisohn Stad...          cc11       en   
56   Too often a beginning bodybuilder has to do hi...          ce01       en   
199  St. Johns , Mich. , April 19 . -- A jury of se...          ca21       en   
188  Knowing specifically what the many feed additi...          ce27       en   
136  Austin , Texas -- Committee approval of Gov. P...          ca02       en   
29   The most surprising thing about the Twenty-sec...          cb25       en   
196  About 70 North Providence taxpayers made appea...          ca24       en   
114  The presidency : talking and listening Though ...          ca42       en   
126  City Controller Alexander Hemphill charged Tue...          ca09       en   
124  California Democrats this weekend will take th...          cb11       en   
142  Greer Garson , world-fa

## 1.4
Based on your observation from above questions, compare linear interpolation and add-λ smoothing by listing out their pros and cons.

1. In Linear Interpolation method, the articles are distinguished as english or portuguese by a margin of ~17 whereas in add-λ smoothing method the margin of ~25 . We can see that trigram model with add-λ smoothing distiguishes better than the combination of Unigram, bigram and trigram model in linear interpolation.
2. In Linear-interpolation the existance of unigram , bigram models induces more bias than the trigram model. Whereas in Trigram model with add-λ smoothing there is only trigram model present so low bias. The add-λ smoothing method helps the trigram model to fix issue with unseen trigrams which is the disadvantage of trigram model.